# **Why function calling?**

Imagine asking someone to write down important information without giving them a form or any guidelines on the structure. You might get a beautifully crafted paragraph, but extracting specific details like names, dates, or numbers would be tedious! Similarly, trying to get consistent structured data from a generative text model without function calling can be frustrating. You're stuck explicitly prompting for things like JSON output, often with inconsistent and frustrating results.

This is where function calling comes in. Instead of hoping for the best in a freeform text response from a generative model, you can define clear functions with specific parameters and data types. These function declarations act as structured guidelines, guiding the Llama model to structure its output in a predictable and usable way. No more parsing text responses for important information!

Think of it like teaching Llama to speak the language of your applications. Need to retrieve information from a database? Define a search_db function with parameters for search terms. Want to integrate with a weather API? Create a get_weather function that takes a location as input. Function calling bridges the gap between human language and the structured data needed to interact with external systems.

# **Objectives**

In this tutorial, you will learn how to use either OpenAI SDK or Vertex AI SDK in Python to make function calls via the Llama 4 Maverick fully managed model on Vertex AI.
 See here for more info on using the [OpenAI SDK with Vertex](https://cloud.google.com/vertex-ai/generative-ai/docs/migrate/openai/overview#:~:text=The%20Chat%20Completions%20API%20works,the%20Google%20Gen%20AI%20SDK.), as well as recommendations on when to use OpenAI SDK vs. Vertex AI SDK.

We will use a currency exchange function as an example, you can replace it with another function with the right functionality for you.
This tutorial is based on this Vertex AI codelab: https://codelabs.developers.google.com/codelabs/gemini-function-calling

# **Setup and Relevant Links**
Llama on Vertex AI (fully managed): https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/llama

Official docs from Vertex on tool calling with Llama coming soon.


# **First, with OpenAI's SDK**

Handling imports and setup

In [ ]:
import google.auth
import openai
import json

from google.auth.transport import requests as google_requests

In [ ]:
import os

project_id = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not project_id or project_id == "[your-project-id]":
    project_id = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

location = os.environ.get("GOOGLE_CLOUD_REGION", "us-east5")

# run gcloud auth print-access-token from terminal to get this
access_token = ""

# Set up OpenAI client
base_url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/endpoints/openapi"
client = openai.OpenAI(base_url=base_url, api_key=access_token)

define the function implementation

In [ ]:
def get_exchange_rate(currency_date, currency_from, currency_to):
    """Get the exchange rate for currencies between countries"""
    try:
        url = f"https://api.frankfurter.app/{currency_date}"
        params = {
            "from": currency_from,
            "to": currency_to
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        return json.dumps(response.json())
    except Exception as e:
        return f"Error fetching exchange rate: {str(e)}"

set up the function declaration for the model

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_exchange_rate",
        "description": "Get the exchange rate for currencies between countries",
        "parameters": {
            "type": "object",
            "properties": {
                "currency_date": {
                    "type": "string",
                    "description": "A date in YYYY-MM-DD format or 'latest'"
                },
                "currency_from": {
                    "type": "string",
                    "description": "The currency to convert from in ISO 4217 format"
                },
                "currency_to": {
                    "type": "string",
                    "description": "The currency to convert to in ISO 4217 format"
                }
            },
            "required": ["currency_from", "currency_to"]
        }
    }
}]

set up the client, generate a tool call and execute the tool call
Enter a query into the text field to get started, eg. "100 usd to eur"

In [ ]:
def main():
    # Set up client
    client = openai.OpenAI(base_url=base_url, api_key=access_token)

    # Step 1: Send the user query to the model
    user_query = input("Enter your currency exchange query: ")

    completion = client.chat.completions.create(
        model="meta/llama-4-maverick-17b-128e-instruct-maas",
        messages=messages,
        tools=tools,
        temperature=0.1,
        max_tokens=512,
        tool_choice="auto"
    )

    # Step 2: Check if the model called a function
    if completion.choices[0].message.tool_calls:
        # Step 3: Execute the function
        for tool_call in completion.choices[0].message.tool_calls:
            if tool_call.function.name == "get_exchange_rate":
                args = json.loads(tool_call.function.arguments)
                currency_date = args.get("currency_date", "latest")
                currency_from = args.get("currency_from")
                currency_to = args.get("currency_to")
                result = get_exchange_rate(currency_date, currency_from, currency_to)

                # Add model's response with function call
                messages.append(completion.choices[0].message)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                })

                # Get final response from model
                final_response = client.chat.completions.create(
                    model="meta/llama-4-maverick-17b-128e-instruct-maas",
                    messages=messages,
                    temperature=0.1
                )

                print("Final response:", final_response.choices[0].message.content)
    else:
        print("Direct response:", completion.choices[0].message.content)

if __name__ == "__main__":
    main()

Enter your currency exchange query: 100 usd to eur
Direct response: The exchange rate for 100 USD to EUR is 88.645 EUR.


# **Now with Vertex SDK**

Handling imports and setup

In [ ]:
import json
import requests
import vertexai
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
    Content,
    Part
)

# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Initialize Vertex AI
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

define the function implementation

In [ ]:
def get_exchange_rate(currency_date, currency_from, currency_to):
    """Get the exchange rate for currencies between countries"""
    try:
        url = f"https://api.frankfurter.app/{currency_date}"
        params = {
            "from": currency_from,
            "to": currency_to
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        return {"error": str(e)}

set up the function declaration for the model

In [ ]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
        "type": "object",
        "properties": {
            "currency_date": {
                "type": "string",
                "description": "A date in YYYY-MM-DD format or 'latest'"
            },
            "currency_from": {
                "type": "string",
                "description": "The currency to convert from in ISO 4217 format"
            },
            "currency_to": {
                "type": "string",
                "description": "The currency to convert to in ISO 4217 format"
            }
        },
        "required": ["currency_from", "currency_to"]
    },
)

exchange_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)


set up the client, generate a tool call and execute the tool call
Enter a query into the text field to get started, eg. "100 usd to eur"

In [ ]:
model = GenerativeModel(
    "llama-4-maverick-17b-128e-instruct-maas",
    generation_config=GenerationConfig(temperature=0.1),
    tools=[exchange_tool],
)

def main():
    user_query = input("Enter your currency exchange query: ")

    # Start chat and send user query
    chat = model.start_chat()
    response = chat.send_message(user_query)

    # Check for function calls
    if response.candidates[0].function_calls:
        for function_call in response.candidates[0].function_calls:
            if function_call.name == "get_exchange_rate":
                # Extract arguments
                args = function_call.args
                currency_date = args.get("currency_date", "latest")
                currency_from = args.get("currency_from")
                currency_to = args.get("currency_to")

                # Call the function
                result = get_exchange_rate(currency_date, currency_from, currency_to)

                # Send function result back to model
                final_response = chat.send_message(
                    Content(
                        role="function",
                        parts=[
                            Part.from_function_response(
                                name="get_exchange_rate",
                                response=result
                            )
                        ]
                    )
                )

                print("Final response:", final_response.text)
    else:
        print("Direct response:", response.text)

if __name__ == "__main__":
    main()


Enter your currency exchange query: 100 usd to eur
Final response: The exchange rate for 100 USD to EUR on the latest date is 88.645 EUR.


# **Congrats and conclusion**

Leveraging function calling via Llama 4 in Vertex AI, you've successfully built a generative AI pipeline that uses the OpenAI and/or Vertex AI SDK! Users can ask about exchange rates, and the system will fetch the latest data from an external API and respond with an answer.

Given a prompt from an end-user, Llama takes care of selecting the appropriate function, extracting parameters from the prompt, and returning a structured data object for you to make an external API call.


# **Cleanup**
You can perform the following cleanup to avoid incurring charges to your Google Cloud account for the resources used in this codelab:
*   To avoid unnecessary Google Cloud charges, use the Google Cloud console to  delete your project if you do not need it.
*   If you want to disable the APIs for Vertex AI, navigate to the Vertex AI API Service Details page and click Disable API and confirm.
